# polyglot-ko finetuning 실습

230925

- [ref1](https://github.com/Beomi/KoAlpaca#gradio-%EC%83%98%ED%94%8C---fullfinetune--8bit--%EC%B6%94%EC%B2%9C-)
- [ref2](https://github.com/youngmook/aihub)

# 4bit quantization으로 Polyglot-ko 12.8B QLoRA 학습하기

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="400" class="center"/>
</center>

- Colab Free (T4 GPU)에서, 한국어 언어 모델 중 가장 큰 크기 모델인 Polyglot-ko 12.8B 모델을 QLoRA로 파인튜닝 해 봅시다.

In [1]:
!nvidia-smi

Mon Sep 25 06:44:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 2) 데이터셋 준비

### 2.1) 데이터셋: KoAlpaca v1.1a
- [ref](https://github.com/Beomi/KoAlpaca#%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B-v11-%EC%A0%9C%EC%9E%91-%EB%B0%A9%EB%B2%95)

In [3]:
from datasets import load_dataset
from pprint import pprint

data_koalpaca = load_dataset("beomi/KoAlpaca-v1.1a", split="train")

print(data_koalpaca)

# data 포맷 변환
data_koalpaca = data_koalpaca.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)
pprint(data_koalpaca[0]['text'])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'output', 'url'],
    num_rows: 21155
})


Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

('### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n'
 '\n'
 '### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n'
 '\n'
 '식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. '
 '고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 '
 '부분입니다.\n'
 '\n'
 ' 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n'
 '\n'
 '고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. '
 '다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<|endoftext|>')


### 2.2) 데이터셋: 구름 데이터셋 v2
- [ref](https://github.com/nlpai-lab/KULLM#%EA%B5%AC%EB%A6%84-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B-v2)

In [4]:
from datasets import load_dataset

data_kullm_v2 = load_dataset("nlpai-lab/kullm-v2", split="train")
print(data_kullm_v2)

# data 포맷 변환
data_kullm_v2 = data_kullm_v2.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)
pprint(data_kullm_v2[0]['text'])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['id', 'instruction', 'input', 'output'],
    num_rows: 152630
})


Map:   0%|          | 0/152630 [00:00<?, ? examples/s]

('### 질문: 3원색이란 무엇인가요?\n'
 '\n'
 '### 답변: 세 가지 기본 색은 빨강, 파랑, 노랑입니다. 이 색은 다른 색을 혼합하여 만들 수 없고 다른 모든 색은 다양한 비율로 '
 '조합하여 만들 수 있기 때문에 원색이라고 부릅니다. 빛에 사용되는 첨가제 색상 시스템에서 원색은 빨강, 녹색, '
 '파랑(RGB)입니다.<|endoftext|>')


### 2.3) 데이터셋: KO-Platypus
- [ref](https://github.com/Marker-Inc-Korea/KO-Platypus?fbclid=IwAR1rV_woLbLZ7zglBYO7RVLI3-5jTykqXxS7sbjWG_Pys8nvAhJB5edtxP0#datasets)

In [5]:
from datasets import load_dataset

data_KO_Platypus = load_dataset("kyujinpy/KOpen-platypus", split="train")
print(data_KO_Platypus)

# data 포맷 변환
data_KO_Platypus = data_KO_Platypus.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)
print(data_KO_Platypus[0]['text'])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/24926 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 24926
})


Map:   0%|          | 0/24926 [00:00<?, ? examples/s]

### 질문: 보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.

### 답변: 모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려면 스피너가 $A$와 $B$에 착륙할 확률을 1$에서 빼야 합니다. 이를 방정식으로 쓸 수 있습니다: $P(C) = 1 - P(A) - P(B)$. P(A) = \frac{1}{3}$, $P(B) = \frac{5}{12}$라는 것을 알고 있으므로 이 값을 방정식에 대입하여 단순화할 수 있습니다. 결과는 다음과 같습니다: P(C) = 1 - \frac{1}{3} - frac{5}{12} = \frac{12}{12} - frac{4}{12} - frac{5}{12} = \frac{3}{12}$. 분자와 분모를 $3$로 나누면 이 분수를 줄일 수 있습니다: P(C) = \frac{1}{4}$입니다.<|endoftext|>


## 2.4) 데이터셋 선택
1) KoAlpaca v1.1a
2) 구름 데이터셋 v2
3) KO-Platypus
4) 1)+2)+3)

In [7]:
# data = data_kullm_v2
# data = data_koalpaca
# data = data_KO_Platypus

# or 세 데이터셋 병합
from datasets import Dataset
data_list = data_koalpaca['text'] + data_kullm_v2['text'] + data_KO_Platypus['text']
list_dict = []
for tmp in data_list:
    tmp_dict = {}
    tmp_dict['text'] = tmp
    list_dict.append(tmp_dict)
data = Dataset.from_list(list_dict, split="train")

print(data)

Dataset({
    features: ['text'],
    num_rows: 198711
})


## 쪼개진 모델 로드

- 원래는 단일 파일이기도 하지만, 작은 파일(약 1GB)로 쪼개서 개별로 로드한 레포를 쓰면 RAM이 터지지 않습니다.

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# model_id = "EleutherAI/polyglot-ko-12.8b"  # Orininal polyglot
model_id = "beomi/polyglot-ko-12.8b-safetensors"  # safetensors 컨버팅된 레포
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

## 텍스트 데이터만 tokenize

In [9]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)
print(data[0]['text'])

Map:   0%|          | 0/198711 [00:00<?, ? examples/s]

### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 

식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.

 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 

고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<|endoftext|>


PEFT를 통해 `prepare_model_for_kbit_training`로 Low bit 학습을 준비해줍시다.

In [10]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [11]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [12]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,  # Lora scaling, 64, 128
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"  # SEQ_CLS, SEQ_2_SEQ_LM, TOKEN_CLS
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424


In [13]:
!nvidia-smi

Mon Sep 25 06:57:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    51W / 400W |   9301MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 학습하기!

- 이번 예제에서는 22k개의 아주아주 일부분인 100개 스텝만 학습해봅시다.

In [14]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=50, ## 초소량만 학습: 50 step만 학습. 약 4분정도 걸립니다.
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=5, training_loss=2.3267280578613283, metrics={'train_runtime': 13.7058, 'train_samples_per_second': 0.73, 'train_steps_per_second': 0.365, 'total_flos': 684162231459840.0, 'train_loss': 2.3267280578613283, 'epoch': 0.0})

In [15]:
print("wow")

wow


In [16]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [17]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1539: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `

tensor([[   6,    6,    6, 2438,   29, 1832, 4770,  272,   34, 4253,   29, 1832,
          296, 8678,  270,  750,  283,   15, 2175,  270]])

In [18]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

## 몇 가지 팁

- 만약 학습이 충분히 되지 않으면 `<|endoftext|>` 토큰이 잘 생성되지 않을 수 있습니다
- 이럴떈 충분히 긴 `max_new_tokens`를 준 뒤, `###`으로 잘라서 써보세요. ex) `output.split('###')[0]`
- 아래 결과는 실제 위 학습된(50step, 100개 샘플) 모델의 결과물입니다.
- 생성시에 속도가 꽤 느립니다. 1-2tokens/s 정도라 256토큰 생성시 약 ~3분 시간이 소요됩니다.

In [19]:
gen('건강하게 살기 위한 세 가지 방법은?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 1. 몸을 자주 움직이기 - 운동을 계속 하는 것입니다. - 매일 5분이라도 걷거나 30분이라도 자전거를 타는 등 계속 움직이시면 됩니다. - 운동을 하는 것은 몸에 항상성 즉, 항상 균형을 유지하려고 하는 힘이 작용하는데, 균형이 깨지게 되면 그 유지하려는 힘이 작용하게 됩니다. 그래서 균형을 잡아주어 항상성을 유지하는 힘이 작용하게 됨으로써 우리가 건강하게 살 수 있습니다. 2. 깨끗한 음식 먹기 - 채식을 많이 하시기 바랍니다. - 채소와 채소 잎, 곡식, 곡식 껍데기에는 우리 몸을 중화시키는 영양소들이 있습니다. 이 영양소들로 인해 우리 몸은 산성이 되는 것을 막아주게 됩니다. 그래서 우리가 좀 더 건강하게 살 수 있습니다. 3. 깨끗한 물 마시기 - 깨끗한 물 속에는 각종 세균과 불순물이 들어 있습니다. 그래서 건강하게 살기 위해서는 수돗물을 받고 정수로 걸러서 하루에 2리터 정도는 마셔야 합니다. - 그리고 깨끗한 물 속의 무기물로 인하여 우리는 건강하게 살 수 있습니다. 물은 하루


`### 질문: 건강하게 살기 위한 세 가지 방법은?`

`### 답변: 첫 번째 방법은 과로하지 않는 것입니다. 두 번째 방법은 물을 많이 마시고, 신선한 음식을 먹는 것입니다. 마지막 방법은 정기적으로 운동을 하는 것입니다. 이러한 세 가지 방법은 규칙적이고 건강하게 사는 데 도움이 됩니다. 하지만 이러한 방법은 누구나 알고 있습니다. 건강한 삶을 살기 위한 다른 방법은 없을까요? 그것은 당신이 가장 만족하지 않는 어떤 것을 추구하는 것 입니다. 예를 들어, 만약 당신이 너무 많이 먹는다고 생각한다면, 당신은 식사량을 줄이는 것이 도움이 될 것입니다. 이러한 연습은 또한 당신의 기분을 더 좋게 해줍니다. 이렇게 당신의 기분이 아주 좋아질 때 당신은 건강한 삶을 살고 있는 것입니다. 이러한 것이 인생에서 가장 만족감을 주는 방식이며, 또한 건강하게 사는 방법입니다. 당신의 건강과 행복을 유지하는 중요한 단계는 어떤 것이 있습니까? 이러한 것을 알아내기 위해 다음 질문에 대해 생각해 보십시오. 그리고 나서 그것들을 적고 목록을 만들어 보십시오. 이 과정은 무엇을 생각하고`

In [20]:
gen('슈카월드가 무엇인가요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 슈카월드가 무엇인가요?

### 답변: 경제/ 주식 쪽의 인기 유튜버입니다. 이분이 말이 굉장히 논리적이서 듣고 있으면 재밌는 말을 많이 합니다. 이분도 유튜브를 시작하신 지는 1년 안됐습니다. 2~4분 정도의 영상을 주로 촬영하십니다. 영상에서 본인이 직접 말하는 게 아니라 친구와 함께 스튜디오에서 촬영을 합니다. 영상도 재밌고 좋지만 영상을 보면 이분의 지식에 놀라고 갑니다. 저는 이분을 굉장히 좋게 보고 있어서 구독자지만 구독할 정도입니다. 구독 해보세요 이분의 영상을 보고 있으면 말빨이 대단하다는 것을 느끼곤 합니다.### 추가영상 ### 슈카월드 추천 영상 1. 금리인상 가능성? (feat. 옐런(feat. 미국 대선)) 2. 달러와 금리의 관계 3. 미국 대선 이후, 경기와 증시 분위기 4. 백신의 공급과 달러에 따른 수혜 업종은?슈카 월드를 추천할려고 처음에 생각이 났을 당시에는 유튜브 댓글에서 슈카월드의 영상을 추천한다는 말 듣고 댓글을 쓰려고 했습니다. 그런데 그때는 제가 영상을 거의 다 봤다고 생각했었고 사실 다 모르고 있


`### 질문: 슈카월드가 무엇인가요?`

`### 답변: 슈카월드는 유튜브 채널이며, 다양한 경제 분야의 주제를 다루고 있습니다. 경제분야에 관심이 있으신 분들은 한번쯤은 들어보셨을 채널일 것입니다. 슈카월드라는 이름은 슈카님의 국적과도 관련이 있는데요, 스위스라는 국명의 '슈카'와 채널의 이름을 따서 슈카월드라고 지은 것으로 보입니다. 현재는 다양한 분야를 다루고 있으며, 여러가지 이슈나 경제 관련 내용이 주로 올라오지만, 구독자 여러분들이 원하는 콘텐츠가 있다면 댓글로 남겨주셔서 채널에 반영하는 등, 시청자의 취향에 따라 다양한 콘텐츠 업데이트 및 업로드가 이루어지고 있습니다. � 인준: 슈카월드는 현재 구독자수 120만 명 이상(!), 조회수는 6600만 뷰 이상을 기록하고 있습니다. 구독자와 조회수의 차이는 컨텐츠를 얼마나 즐겨보는지와 관계가 있는 것으로 보입니다. 슈카월드는 경제관련 내용 이외에도, 여행, 정치, 문화, 역사, 스포츠, 상식, 정보, 시사 등 다양한 분야에 대한 상식, 정보를 다루고 있는 채널입니다. 따라서, 슈카월드를 구독한 많은 분들은`


In [ ]:
gen('공공성을 측정하기위한 방법은?')

`### 질문: 공공성을 측정하기위한 방법은?`

`### 답변: 공공성을 측정하기위한 몇 가지 방법 중 하나는 공공기관의 서비스에 대한 청구 비용을 측정하는 것입니다. 이를 위해 A/B 테스트가 사용됩니다. 이 방법은 청구 비용이 평균 값 A보다 클경우 A의 사용을 요구하고, 청구 비용이 평균 값 A보다 작다면 B의 사용을 요구합니다. 공공 기관의 실제 비용을 측정해 공공성을 측정하는 다른 방법에는 정부에서 지불하는 재정 예산이 있습니다. 이러한 비용은 재정적 지출과 관련한 정보를 공개하는 것이 적절합니다. 이러한 정보는 또한 공공 서비스에 대한 청구 비용 측정에서도 동일하게 적용될 수 있습니다. 이를 통해, 공공기관의 공공성이 측정될 수 있습니다.### 질문: 특정 공공 서비스에 대한 청구 비용이 적절한 경우는?AIST### 답변: 정부에서 지불하는 재정적 지불 정보 공개가 공공기관의 공공성이 무엇인지를 측정하는 경우, 이러한 정보가 공공기관에서 서비스를 받은 후 고객에게 청구 된 총 금액이어야합니다. 만약, 지불 금액을 서비스 비용과 같은 개념으로 이해한다면, 공공기관의 다른 서비스 비용 또한 청구 비용으로 측정`

In [ ]:
gen('주식 시장에서 안정적으로 수익을 얻기 위한 방법은?')

`### 질문: 주식 시장에서 안정적으로 수익을 얻기 위한 방법은?`

`### 답변: 주식시장은 변동성이 높습니다. 하지만, 변동성이 높음에도 불구하고 수익을 얻으려면 장기적으로 봐야하고, 기업의 미래 가치를 분석하고 투자해야 합니다. 기업의 주식을 샀다면 그 기업의 미래 가치가 좋아야 하고 성장해야 합니다. 그래야 꾸준히 높은 주가 상승을 볼 수 있습니다. 하지만, 단기간의 수익을 얻길 원한다면 다른 방법을 권합니다. 이러한 이유로 주식 투자를 하는 많은 개인들은 단타의 형태를 보입니다. 단타는 단기적으로 수익을 얻고 빠져나오는 방법을 의미합니다. 주식을 사고 일정 기간이 지나지 않았음에도 자신의 수익이 났다면 그 수익금을 챙기고 나오는 형태입니다. 단타는 변동성이 높은 주식 시장에서 비교적 짧은 기간 동안 수익을 보고 나오는 방법입니다. 기업에서 꾸준히 수익을 내는 것보다 훨씬 쉬운 만큼 리스크가 큰 방법입니다. 때문에 개인이 주식 투자에서 안정적으로 수익을 얻길 원하신다면 장기적으로 투자를 하시는 것을 권합니다. 꾸준한 수익을 내기 위해 노력하는 기업을 찾으시고 함께 성장하면 됩니다. 이러한 노력은 시간이 걸립니다. 단, 기간적인 부분에서 시간이 오래 걸릴`


In [ ]:
gen('풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?')

`### 질문: 풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?`

`### 답변: 풋옵션과 콜옵션의 차이점은, 우선 둘 다 금융 시장에서 거래가 가능한 파생상품이며 이 옵션들은 거래 방법이 다릅니다. 주식을 사고파는 주식거래와 달리, 이 상품은 당사자 사이에 계약을 통해 거래하게 됩니다. 또한, 주식과 달리, 두 옵션상품 모두 시장 상황에 따른 가격의 상승과 하락에 영향을 받을 수 있습니다. 그리고 두 옵션의 차이점과 더 중요한 차이점은, 옵션은 거래 금액이 고정되지 않습니다. 즉, 주식거래와 같이 계약한 금액에 고정된 금액으로 거래하는 것이 아니라는 의미입니다. 이는 주가가 급변할 수도, 가격 변동 폭이 매우 클 수도 있는 상황에서 주식 거래와 옵션 거래를 비교했을 때, 옵션 거래가 훨씬 더 위험한 이유이기도 합니다. 그래서 일반 개미 투자자는 주식거래나 선물 거래와 같이 일정한 금액이 고정되어 거래되는, 안정성이 더 큰 일반적인 주식거래를 합니다. 일반 개미 투자자가 이러한 주식거래의 일종인 선물과 주식거래의 일종인 옵션거래에서 선택해야 하는 포지션은, 우선 선물거래는 선물 매도 포지션 (자신의 보유 주식을 선물 매도 계약에 포함하여 보유`


In [ ]:
gen('풋옵션 매도와 콜옵션 매수의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?')

`### 질문: 풋옵션 매도와 콜옵션 매수의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?`

`### 답변: 풋옵션과 콜옵션의 매수와 매도 포지션의 정의를 먼저 알아보겠습니다. 풋옵션이란 '풋'이란 판매하는 사람이 파는 금액을 의미하고, '옵션'이란 선택 사항을 의미합니다. 즉, 풋옵션은 '풋'이라고 표기를 해야 하고, 선택 사항이 있음을 의미합니다. 이러한 옵션은 주가에 대해서 행사하는 권리를 부여하는 것으로, 이러한 옵션을 선택하였을 때는 해당 주식과 같은 방향으로 매도하는 것과 구매하는 것이 가능합니다. 예를 들어서, 주가가 하락할 것이라는 기대가 있다면, 이러한 예상으로 인하여 주가가 떨어질 것이기 때문에 일반 매도 포지션으로 주가의 하락에 참가하는 것이 낫지만, 주가가 하락할 때 해당 주식을 일정 부분 상승의 방향으로, 또는 상승의 폭만큼을 보장받고 싶다면, 이러한 옵션을 선택하는 것입니다. 즉, 옵션에서 주가가 상승을 할 때는 콜옵션으로 선택하는 것이고, 주가가 하락하게 될 때는 풋옵션을 선택하게 됩니다. 이러한 선택을 할 경우, 일반적으로 주가의 하락에 대해서 상승의 방향으로 상승폭`


In [ ]:
gen("마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?")

`### 질문: 마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?`

`### 답변: 마진콜이 발생하는 이유는 은행이 고객에게 지급하고자 하는 금액이 발생한 금액보다 많은 경우, 추가로 내야 하는 금액을 계산해서 계좌에서 더 많은 금액을 인출하게 되는 것입니다. 이렇게 되면, 계좌를 관리하는 은행 입장에서는 계좌에서 추가로 꺼내야 하기 때문에, 은행의 신뢰도가 땅으로 떨어지고, 계좌를 사용하는 은행 고객도 추가로 내야 하는 금액에 대한 불안감을 느낄 수 있습니다. 이런 불안감에 대해 잘 보여주는 사례가 영화 마진콜입니다. 따라서, 이런 현상을 막기 위해서는 고객에게 먼저 지급할 금액과 인출하는 금액을 정확하게 파악하는 것이 중요합니다.`



## 모델 저장 & 업로드

In [21]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: Traceback (most recent call last):
  File "/usr/lib/python3.10/getpass.py", line 77, in unix_getpass
    passwd = _raw_input(prompt, stream, input=input)
  File "/usr/lib/python3.10/getpass.py", line 146, in _raw_input
    line = input.readline()
KeyboardInterrupt

During handling of the above exception, anoth

In [22]:
# model.push_to_hub('beomi/qlora-koalpaca-polyglot-12.8b-50step')
# 아래 아이디 부분을 수정해서 쓰세요.
# model.push_to_hub('허깅페이스아이디/qlora-koalpaca-polyglot-12.8b-50step')
model.push_to_hub('keepsteady/qlora-polyglot-12.8b-50step')

ValueError: ignored

- 위 예시코드는 https://huggingface.co/keepsteady/qlora-koalpaca-polyglot-12.8b-50step 에 올라갑니다. 사용하시는 이름으로 바꿔쓰세요.

# Hugginface Hub에 올린 LoRA Weight 저장한것 불러오는 튜토리얼 코드

- HF에 업로드 후, 실제 다운받아 사용시 쓰는 코드 예제입니다.

--> https://github.com/Beomi/KoAlpaca/blob/main/2023_06_08_PEFT%EB%A1%9CLoRA%EB%A1%9C%EB%93%9C.ipynb

In [23]:
!pip -q install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.5 MB/s eta 0:00:00


In [ ]:
## gradio inference example code
# ref: https://github.com/Beomi/KoAlpaca/blob/main/webui/app.py
import gradio as gr
import torch
from transformers import pipeline, AutoModelForCausalLM

# MODEL = "beomi/KoAlpaca-Polyglot-12.8B"
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL,
#     device_map="auto",
#     load_in_8bit=True,
#     revision="8bit",
#     # max_memory=f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'
# )

# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=MODEL,
#     # device=2,
# )


def answer(state, state_chatbot, text):
    messages = state + [{"role": "질문", "content": text}]

    conversation_history = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )

#     ans = pipe(
#         conversation_history + "\n\n### 답변:",
#         do_sample=True,
#         max_new_tokens=512,
#         temperature=0.7,
#         top_p=0.9,
#         return_full_text=False,
#         eos_token_id=2,
#     )

    gened = model.generate(
        **tokenizer(
#             f"### 질문: {x}\n\n### 답변:",
            f"{conversation_history}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )



#     msg = ans[0]["generated_text"]
    msg = tokenizer.decode(gened[0])


    if "###" in msg:
        msg = msg.split("###")[0]

    new_state = [{"role": "이전 질문", "content": text}, {"role": "이전 답변", "content": msg}]

    state = state + new_state
    state_chatbot = state_chatbot + [(text, msg)]

    print(state)
    print(state_chatbot)

    return state, state_chatbot, state_chatbot


with gr.Blocks(css="#chatbot .overflow-y-auto{height:750px}") as demo:
    state = gr.State(
        [
            {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
        ]
    )
    state_chatbot = gr.State([])

    with gr.Row():
        gr.HTML(
            """<div style="text-align: center; max-width: 500px; margin: 0 auto;">
            <div>
                <h1>ChatKoAlpaca 12.8B (v1.1b-chat-8bit)</h1>
            </div>
            <div>
                Demo runs on RTX 3090 (24GB) with 8bit quantized
            </div>
        </div>"""
        )

    with gr.Row():
        chatbot = gr.Chatbot(elem_id="chatbot")

    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="Send a message...").style(
            container=False
        )

    txt.submit(answer, [state, state_chatbot, txt], [state, state_chatbot, chatbot])
    txt.submit(lambda: "", None, txt)

demo.launch(debug=True, share=True, server_name="0.0.0.0")

<ipython-input-24-360b52f46956>:110: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  txt = gr.Textbox(show_label=False, placeholder="Send a message...").style(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1539: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


[{'role': '맥락', 'content': 'KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.'}, {'role': '맥락', 'content': 'ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.'}, {'role': '명령어', 'content': '친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다.'}, {'role': '명령어', 'content': '인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.'}, {'role': '이전 질문', 'content': '오늘 뭐해?'}, {'role': '이전 답변', 'content': ''}]
[('오늘 뭐해?', '')]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'role': '맥락', 'content': 'KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.'}, {'role': '맥락', 'content': 'ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.'}, {'role': '명령어', 'content': '친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다.'}, {'role': '명령어', 'content': '인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.'}, {'role': '이전 질문', 'content': '오늘 뭐해?'}, {'role': '이전 답변', 'content': ''}, {'role': '이전 질문', 'content': 'hello?'}, {'role': '이전 답변', 'content': ''}]
[('오늘 뭐해?', ''), ('hello?', '')]


# Finish!!